# BB84
The BB84 protocol is a basic quantum algorithm for distributing keys.
This is the noise-free version of the algorithm.

In [1]:
# Needed imports and functions
import qiskit
import hashlib 

# import all necessary objects and methods for quantum circuits
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from random import randrange

def print_outcomes_in_reserve(counts): # takes a dictionary variable
    for outcome in counts: # for each key-value in dictionary
        reverse_outcome = ''
        for i in outcome: # each string can be considered as a list of characters
            reverse_outcome = i + reverse_outcome # each new symbol comes before the old symbol(s)
    return reverse_outcome

#Source: awards/teach_me_qiskit_2018/cryptography/Cryptography.ipynb
def SendState(qc1, qc2, qc1_name):
    ''' This function takes the output of a circuit qc1 (made up only of x and 
        h gates and initializes another circuit qc2 with the same state
    ''' 
    
    # Quantum state is retrieved from qasm code of qc1
    qs = qc1.qasm().split(sep=';')[4:-1]

    # Process the code to get the instructions
    for index, instruction in enumerate(qs):
        qs[index] = instruction.lstrip()

    # Parse the instructions and apply to new circuit
    for instruction in qs:
        if instruction[0] == 'x':
            if instruction[5] == '[':
                old_qr = int(instruction[6:-1])
            else:
                old_qr = int(instruction[5:-1])
            qc2.x(qreg[old_qr])
        elif instruction[0] == 'h':
            if instruction[5] == '[':
                old_qr = int(instruction[6:-1])
            else:
                old_qr = int(instruction[5:-1])
            qc2.h(qreg[old_qr])
        elif instruction[0] == 'm': # exclude measuring:
            pass
        else:
            raise Exception('Unable to parse instruction')


In [2]:
n = 24

print('---- ENCODING MESSAGE ----')

# Asja's circuit
qreg = QuantumRegister(n)
creg = ClassicalRegister(n)

asja = QuantumCircuit(qreg, creg, name = 'Asja')

print("Asja's circuit initialized")

# Random message
message = []

for i in range(n):
    bit = randrange(2)
    message.append(bit)

print(f'Message is: {message}')
    
# Encode message
for i, m in enumerate(message):
    if m == 1:
        asja.x(qreg[i])

# Choose basis
asja_basis = []

for i in range(n):
    basis = randrange(2)
    if basis == 0:
        asja_basis.append('Z')
    else:
        asja.h(qreg[i])
        asja_basis.append('X')
        
print(f'Asja basis: {asja_basis}')

print('---- RECEIVING MESSAGE ----')

# Balvis circuit, which recieved the qubits
balvis = QuantumCircuit(qreg, creg, name = 'Balvis')
SendState(asja, balvis, 'Asja')

print("Balvis received qubits")

# Measure
balvis_basis = []

for i in range(n):
    basis = randrange(2)
    
    # Choose measurement basis
    if basis == 0:
        balvis.measure(qreg[i],creg[i])
        balvis_basis.append('Z')
    else: 
        balvis.h(qreg[i])
        balvis.measure(qreg[i],creg[i])
        balvis_basis.append('X')
        
print(f'Balvis basis: {balvis_basis}')

# Run balvis
job = execute(balvis, Aer.get_backend('qasm_simulator'), shots=1)
counts = job.result().get_counts(balvis)
counts = print_outcomes_in_reserve(counts)
received_message = list(map(int, counts))

print(f'Balvis measured the message: {received_message}')

# Sifting

print('---- SIFTING ----')

asja_key = []
balvis_key = []

for i in range(n): 
    # If same basis used, keep part of message
    if asja_basis[i] == balvis_basis[i]:
        asja_key.append(message[i])
        balvis_key.append(received_message[i])
    else:
        pass

print("Parties sifted:")
print(asja_key)
print(balvis_key)

# QBER

print('---- QBER ----')

rounds = len(asja_key) // 3
errors = 0

for i in range(rounds):
    # Choose a random bit
    index = randrange(len(asja_key)) 
    tested_bit = asja_key[index]
    
    print ("Asja selected bit =", index, ", with value = ", tested_bit)
    
    if asja_key[index] != balvis_key[index]:
        errors += 1

    # Remove test bits
    del asja_key[index]
    del balvis_key[index]
    
QBER = errors / rounds
        
print("QBER value = ", QBER)
print("Asja's secret key = ", asja_key)
print("Balvis' secret key = ", balvis_key)

if QBER != 0:
    raise ValueError('QBER not 0, someone intercepted the message...')

# Privacy amplification

print('---- PRIVACY AMP - SHA HASHING ----')

# Random seed
seed = []
for i in asja_key:
    a = randrange(2)
    seed.append(a)

asja_key.append(seed)
balvis_key.append(seed)

# Converting lists to strings
str_key_asja = ' '.join([str(elem) for elem in asja_key])
str_key_balvis = ' '.join([str(elem) for elem in balvis_key])

# First bit determines hash fx
if asja_key[0] == 0:
    result = hashlib.sha256(str_key_asja.encode())
    print("Encoded using SHA256:", result.hexdigest())
else:
    result = hashlib.sha3_256(str_key_asja.encode())
    print("Encoded using SHA3 256:", result.hexdigest())

print('Encoded key in binary:')
bin(int(result.hexdigest(), 16))[2:]

---- ENCODING MESSAGE ----
Asja's circuit initialized
Message is: [1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1]
Asja basis: ['Z', 'X', 'X', 'X', 'X', 'X', 'Z', 'X', 'X', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'X', 'X', 'X', 'Z']
---- RECEIVING MESSAGE ----
Balvis received qubits
Balvis basis: ['X', 'X', 'Z', 'X', 'Z', 'X', 'X', 'Z', 'X', 'X', 'X', 'X', 'Z', 'X', 'X', 'Z', 'Z', 'X', 'X', 'Z', 'Z', 'X', 'X', 'Z']
Balvis measured the message: [1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1]
---- SIFTING ----
Parties sifted:
[0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1]
[0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1]
---- QBER ----
Asja selected bit = 2 , with value =  1
Asja selected bit = 0 , with value =  0
Asja selected bit = 10 , with value =  1
Asja selected bit = 7 , with value =  0
QBER value =  0.0
Asja's secret key =  [0, 0, 1, 1, 1, 0, 0, 1, 1]
Balvis' secret key =  [0, 0, 1, 1, 1, 0, 0, 1, 1]
---- PRIVACY AMP - SHA HASHING --

'101110000100010100111101110001111101100001100010101110100100000000100101110001100111111100000111110100010001101101101100000001100010010101111111110101111010001000110001000011011110010001111110110111001100010000000101101010011011111011101101001011001101100'